In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import load
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout, Conv1D, MaxPooling1D, LSTM, Dense

data = load('/content/drive/MyDrive/Footprints Datasets/real.npz')
x, y = data['x'], data['y']
data = load('/content/drive/MyDrive/Footprints Datasets/fake.npz')

derived_data = load('2my_data.npz')
derived_data3 = load('4my_data.npz')


x_fake,y_fake=data['x'],data['y']
# x[x > 1e9] = 1e9
# x_fake[x_fake>1e9]=1e9

In [ ]:
print(derived_data.shape)
print(train_real_data.shape)

(3, 1998, 101)
(525, 1998, 101, 1)


In [9]:
from sklearn.preprocessing import MinMaxScaler
def normalize_array(arr):
    try:
        my_inf = np.isinf(arr)
        arr[np.isinf(arr)] = 0
        scaler = MinMaxScaler()
        normalized_arr = scaler.fit_transform(arr)
        normalized_arr[my_inf] = 1
        return normalized_arr
    except ValueError as e:
        # if 'contains infinity' in str(e) or 'too large' in str(e):
        #     return np.zeros_like(arr)
        # else:
            raise e

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

for i in range(x.shape[0]):
    x[i] = normalize_array(x[i])
    x_fake[i]= normalize_array(x_fake[i])

x = np.expand_dims(x, axis=-1)
x_fake = np.expand_dims(x_fake,axis=-1)
train_end_index = int(0.7 * x.shape[0])
test_start_index = int(0.7 * x.shape[0])
test_end_index = int(0.7 * x.shape[0]) + int(0.3 * x.shape[0])

# Extract train and test data
train_real_data = x[:train_end_index]
test_real_data = x[test_start_index:test_end_index]
train_fake_data=x_fake[:train_end_index]
test_fake_data=x_fake[test_start_index:test_end_index]

train_real_labels = np.ones(train_real_data.shape[0])
test_real_labels = np.ones(test_real_data.shape[0])

train_fake_labels = np.zeros(train_real_data.shape[0])
test_fake_labels = np.zeros(test_real_data.shape[0])

train_data = np.empty(
    (2 * train_real_data.shape[0], train_real_data.shape[1], train_real_data.shape[2],train_real_data.shape[3]),
                               dtype=train_real_data.dtype)
test_data = np.empty(
    (2 * test_real_data.shape[0], test_real_data.shape[1], test_real_data.shape[2],test_real_data.shape[3]),
                               dtype=test_real_data.dtype)

train_data[::2] = train_real_data
train_data[1::2] = train_fake_data

test_data[::2] = test_real_data
test_data[1::2] = test_fake_data

train_labels = np.empty((2 * train_real_data.shape[0] ))
train_labels[::2] = train_real_labels
train_labels[1::2] = train_fake_labels

test_labels = np.empty((2 * test_real_data.shape[0] ))
test_labels[::2] = test_real_labels
test_labels[1::2] = test_fake_labels

derived_data = np.array([derived_data[key] for key in derived_data])
for i in range(3):
    derived_data[i] = normalize_array(derived_data[i])
derived_data = np.expand_dims(derived_data, axis=-1)

derived_data3 = np.array([derived_data3[key] for key in derived_data3])
for i in range(3):
    derived_data3[i] = normalize_array(derived_data3[i])
derived_data3 = np.expand_dims(derived_data3, axis=-1)

train_data = np.concatenate((train_data, derived_data), axis=0)

testing2 = np.zeros(3)
train_labels = np.concatenate((train_labels, testing2), axis=0)

train_data = np.concatenate((train_data, derived_data3), axis=0)
testing3 = np.array([1,0,0])
train_labels = np.concatenate((train_labels, testing3), axis=0)


# train_data = np.vstack((train_real_data, train_fake_data))
# train_labels = np.concatenate((np.ones(train_real_data.shape[0]), np.zeros(train_fake_data.shape[0])))

# indices = np.random.permutation(train_data.shape[0])

# train_data = train_data[indices]
# train_labels = train_labels[indices]
# print(train_labels.size)
# print(np.count_nonzero(train_labels))

# test_data[::2] = test_real_data
# test_data[1::2] = test_fake_data
# test_labels = np.empty((2 * test_real_data.shape[0] ))
# test_labels[::2] = test_real_labels
# test_labels[1::2] = test_fake_labels



In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(1998, 101, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Adjust the learning rate as needed
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
print(model.summary())

In [11]:
train_data_tensor = tf.convert_to_tensor(train_data, dtype=tf.float32)
train_labels_tensor = tf.convert_to_tensor(train_labels, dtype=tf.float32)
test_data_tensor = tf.convert_to_tensor(test_data, dtype=tf.float32)
test_labels_tensor = tf.convert_to_tensor(test_labels, dtype=tf.float32)

In [ ]:
hist = model.fit(train_data_tensor, train_labels_tensor, epochs=40, validation_data=(test_data_tensor, test_labels_tensor))


In [ ]:
yhat = model.predict(test_data_tensor)
# print(yhat[::2])
yhat = [1 if prediction > 0.8 else 0 for prediction in yhat]
accuracy = np.mean(yhat == test_labels)
print(accuracy)

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [16]:
import numpy as np
from matplotlib.pyplot import imread


class SteganographyException(Exception):
    pass

class LSBSteg():
    def __init__(self, im):
        self.image = im
        self.height, self.width, self.nbchannels = im.shape
        self.size = self.width * self.height

        self.maskONEValues = [1,2,4,8,16,32,64,128]
        #Mask used to put one ex:1->00000001, 2->00000010 .. associated with OR bitwise
        self.maskONE = self.maskONEValues.pop(0) #Will be used to do bitwise operations

        self.maskZEROValues = [254,253,251,247,239,223,191,127]
        #Mak used to put zero ex:254->11111110, 253->11111101 .. associated with AND bitwise
        self.maskZERO = self.maskZEROValues.pop(0)

        self.curwidth = 0  # Current width position
        self.curheight = 0 # Current height position
        self.curchan = 0   # Current channel position

    def put_binary_value(self, bits): #Put the bits in the image
        for c in bits:
            val = list(self.image[self.curheight,self.curwidth]) #Get the pixel value as a list
            if int(c) == 1:
                val[self.curchan] = int(val[self.curchan]) | self.maskONE #OR with maskONE
            else:
                val[self.curchan] = int(val[self.curchan]) & self.maskZERO #AND with maskZERO

            self.image[self.curheight,self.curwidth] = tuple(val)
            self.next_slot() #Move "cursor" to the next space

    def next_slot(self):#Move to the next slot were information can be taken or put
        if self.curchan == self.nbchannels-1: #Next Space is the following channel
            self.curchan = 0
            if self.curwidth == self.width-1: #Or the first channel of the next pixel of the same line
                self.curwidth = 0
                if self.curheight == self.height-1:#Or the first channel of the first pixel of the next line
                    self.curheight = 0
                    if self.maskONE == 128: #Mask 1000000, so the last mask
                        raise SteganographyException("No available slot remaining (image filled)")
                    else: #Or instead of using the first bit start using the second and so on..
                        self.maskONE = self.maskONEValues.pop(0)
                        self.maskZERO = self.maskZEROValues.pop(0)
                else:
                    self.curheight +=1
            else:
                self.curwidth +=1
        else:
            self.curchan +=1

    def read_bit(self): #Read a single bit int the image
        val = self.image[self.curheight,self.curwidth][self.curchan]
        val = int(val) & self.maskONE
        self.next_slot()
        if val > 0:
            return "1"
        else:
            return "0"

    def read_byte(self):
        return self.read_bits(8)

    def read_bits(self, nb): #Read the given number of bits
        bits = ""
        for i in range(nb):
            bits += self.read_bit()
        return bits

    def byteValue(self, val):
        return self.binary_value(val, 8)

    def binary_value(self, val, bitsize): #Return the binary value of an int as a byte
        binval = bin(val)[2:]
        if len(binval) > bitsize:
            raise SteganographyException("binary value larger than the expected size")
        while len(binval) < bitsize:
            binval = "0"+binval
        return binval

    def encode_text(self, txt):
        l = len(txt)
        binl = self.binary_value(l, 16) #Length coded on 2 bytes so the text size can be up to 65536 bytes long
        self.put_binary_value(binl) #Put text length coded on 4 bytes
        for char in txt: #And put all the chars
            c = ord(char)
            self.put_binary_value(self.byteValue(c))
        return self.image

    def decode_text(self):
        ls = self.read_bits(16) #Read the text size in bytes
        l = int(ls,2)
        i = 0
        unhideTxt = ""
        while i < l: #Read all bytes of the text
            tmp = self.read_byte() #So one byte
            i += 1
            unhideTxt += chr(int(tmp,2)) #Every chars concatenated to str
        return unhideTxt

    def encode_image(self, imtohide):
        w = imtohide.width
        h = imtohide.height
        if self.width*self.height*self.nbchannels < w*h*imtohide.channels:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        binw = self.binary_value(w, 16) #Width coded on to byte so width up to 65536
        binh = self.binary_value(h, 16)
        self.put_binary_value(binw) #Put width
        self.put_binary_value(binh) #Put height
        for h in range(imtohide.height): #Iterate the hole image to put every pixel values
            for w in range(imtohide.width):
                for chan in range(imtohide.channels):
                    val = imtohide[h,w][chan]
                    self.put_binary_value(self.byteValue(int(val)))
        return self.image


    def decode_image(self):
        width = int(self.read_bits(16),2) #Read 16bits and convert it in int
        height = int(self.read_bits(16),2)
        unhideimg = np.zeros((width,height, 3), np.uint8) #Create an image in which we will put all the pixels read
        for h in range(height):
            for w in range(width):
                for chan in range(unhideimg.channels):
                    val = list(unhideimg[h,w])
                    val[chan] = int(self.read_byte(),2) #Read the value
                    unhideimg[h,w] = tuple(val)
        return unhideimg

    def encode_binary(self, data):
        l = len(data)
        if self.width*self.height*self.nbchannels < l+64:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        self.put_binary_value(self.binary_value(l, 64))
        for byte in data:
            byte = byte if isinstance(byte, int) else ord(byte) # Compat py2/py3
            self.put_binary_value(self.byteValue(byte))
        return self.image

    def decode_binary(self):
        l = int(self.read_bits(64), 2)
        output = b""
        for i in range(l):
            output += chr(int(self.read_byte(),2)).encode("utf-8")
        return output

def encode(image: np.ndarray, message: str) -> np.array:
    # steg = LSBSteg(cv2.imread(filepath))
    steg = LSBSteg(image)
    img_encoded = steg.encode_text(message)
    return img_encoded

def decode(encoded: np.array) -> str:
    steg = LSBSteg(encoded)
    return steg.decode_text()

In [ ]:
# my_real = np.array([my_real[key] for key in my_real])
# for i in range(3):
#     my_real[i] = normalize_array(my_real[i])
# my_real = np.expand_dims(my_real, axis=-1)
print(my_real.shape)

(3, 1998, 101, 1)


In [24]:
import numpy as np
# from LSBSteg import decode
import requests
import tensorflow as tf

team_id = "Jt4hTHH"
# team_id = "a3333333"
# 3.70.97.142
api_base_url = "http://3.70.97.142:5000/eagle"
global_response = {}
counter = 1

def init_eagle(team_id):
    '''
    In this fucntion you need to hit to the endpoint to start the game as an eagle with your team id.
    If a sucessful response is returned, you will recive back the first footprints.
    '''

    payload = {
        "teamId": team_id
    }

    response = requests.post(api_base_url + f'/start', json=payload)
    if response.status_code == 200 or response.status_code == 201:
        response_data = response.json()
        footprints = response_data['footprint']
        # print("footprints received")
        return footprints
    pass

def select_channel(footprint):
    '''
    According to the footprint you recieved (one footprint per channel)
    you need to decide if you want to listen to any of the 3 channels or just skip this message.
    Your goal is to try to catch all the real messages and skip the fake and the empty ones.
    Refer to the documentation of the Footprints to know more what the footprints represent to guide you in your approach.
    '''
    # Use the model to predict the channel to listen to
    footprint_tf = tf.convert_to_tensor(footprint, dtype=tf.float32)
    prediction = model.predict(footprint_tf)
    # print("current prediction: ")
    # print(prediction)
    return prediction


def skip_msg(team_id):
    '''
    If you decide to NOT listen to ANY of the 3 channels then you need to hit the end point skipping the message.
    If sucessful request to the end point , you will expect to have back new footprints IF ANY.
    '''
    payload = {
        "teamId": team_id
    }

    response = requests.post(api_base_url + f'/skip-message', json=payload)
    # print(response)
    if response.status_code == 200 or response.status_code == 201:
        response_data = response.json()
        # print(response_data)
        # print("Message skipped")
        footprints = response_data['nextFootprint']
        return footprints
    else:
        print(response.text)
        # print(f"Error: Failed to skip message. Status code: {response.status_code}")
        return None
    pass

def request_msg(team_id, channel_id):
    '''
    If you decide to listen to any of the 3 channels then you need to hit the end point of selecting a channel to hear on (1,2 or 3)
    '''

    payload = {
        "teamId": team_id,
        "channelId": int(channel_id)
    }

    response = requests.post(api_base_url + f'/request-message', json=payload)
    # print(response)
    if response.status_code == 200 or response.status_code == 201:
        response_data = response.json()
        # print("requested message " + str(channel_id))
        encoded_message = response_data['encodedMsg']
        return encoded_message
    pass

def submit_msg(team_id, decoded_msg):
    '''
    In this function you are expected to:
        1. Decode the message you requested previously
        2. call the api end point to send your decoded message
    If sucessful request to the end point , you will expect to have back new footprints IF ANY.
    '''

    payload = {
        "teamId": team_id,
        "decodedMsg": decoded_msg
    }

    response = requests.post(api_base_url + f'/submit-message', json=payload)
    # print(response)
    if response.status_code == 200 or response.status_code == 201:
        response_data = response.json()
        next_footprint = response_data['nextFootprint']
        return next_footprint
    else:
        print(response.text)
        # print(f"Error: Failed to submit message. Status code: {response.status_code}")
        return None

def end_eagle(team_id):
    '''
    Use this function to call the api end point of ending the eagle  game.
    Note that:
    1. Not calling this fucntion will cost you in the scoring function
    '''
    payload = {
        "teamId": team_id
    }

    response = requests.post(api_base_url + f'/end-game', json=payload)
    # print(response)
    if response.status_code == 200 or response.status_code == 201:
        global_response = response
        response_data = response.text
        # print(response_data)
        return response_data
    pass

def submit_eagle_attempt(team_id):
    '''
     Call this function to start playing as an eagle.
     You should submit with your own team id that was sent to you in the email.
     Remeber you have up to 15 Submissions as an Eagle In phase1.
     In this function you should:
        1. Initialize the game as fox
        2. Solve the footprints to know which channel to listen on if any.
        3. Select a channel to hear on OR send skip request.
        4. Submit your answer in case you listened on any channel
        5. End the Game
    '''
    current_message = ""
    end_of_text = ""
    # data2 = load('/content/drive/MyDrive/Footprints Datasets/real.npz')
    footprints = init_eagle(team_id)
    # footprints = {"1": data2['x'][0], "2": data2['x'][1], "3": data2['x'][2]}



    probability = [0, 0, 0]
    counter = 15
    while footprints is not None:
        np.savez(str(counter) + "my_data.npz", **footprints)
        counter += 1

        footprints = np.array([footprints[key] for key in footprints])
        for i in range(3):
            footprints[i] = normalize_array(footprints[i])
        footprints = np.expand_dims(footprints, axis=-1)

        probability = select_channel(footprints)

        if(max(probability) > 0.2):
            channel_id = np.argmax(probability) + 1
            img = request_msg(team_id, channel_id)
            img = np.array(img)

            decoded_message = decode(img)
            print("decoded message = " + decoded_message)
            footprints = submit_msg(team_id, decoded_message)
        else:
            footprints = skip_msg(team_id)

    ret = end_eagle(team_id)
    print(ret)
    pass


submit_eagle_attempt(team_id)


1/1 [==============================] - 0s 18ms/step
decoded message = xels hold the key.
End of message reached
Game ended successfully with a score of 66.68525062500001. Which is not higher than your highest score.


In [18]:
end_eagle(team_id)

Game ended successfully with a score of 14.902263455056179. Which is not higher than your highest score.


'Game ended successfully with a score of 14.902263455056179. Which is not higher than your highest score.'

In [ ]:
from sklearn.preprocessing import StandardScaler

def normalize2(arr):
  scaler = StandardScaler()
  normalized_array = scaler.fit_transform(arr)
  return normalized_array

def sigmoid(x):
  return 1 / (1 + np.exp(-x))